In [34]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_curve,roc_auc_score,log_loss
import matplotlib.pyplot as plt  
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.compose import make_column_selector,make_column_transformer

In [35]:
medical_df = pd.read_csv(r'..\Cases\Medical Cost Personal\insurance.csv')
medical_df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [36]:
# spliting
X= medical_df.drop(columns=['charges'])
y= medical_df['charges']
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=25)

In [37]:
# Encoding
ohe=OneHotEncoder(sparse=False,drop='first')
from sklearn.compose import make_column_selector
ct1=make_column_transformer((ohe, make_column_selector(dtype_include='object')), remainder='passthrough',verbose_feature_names_out=False)
ct1

ColumnTransformer(remainder='passthrough',
                  transformers=[('onehotencoder',
                                 OneHotEncoder(drop='first', sparse=False),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x000002A15ED2FA90>)],
                  verbose_feature_names_out=False)

1. KNN without scaling

In [38]:
x_train_encode=ct1.fit_transform(X_train)
x_test_encode=ct1.transform(X_test)

In [ ]:
# calculate for 1,3,5,7,9
from sklearn.metrics import r2_score

scores=[]
for i in range(1,10,2):
    knn=KNeighborsRegressor(n_neighbors=i)
    knn.fit(x_train_encode,y_train)
    y_pred=knn.predict(x_test_encode)
    scores.append([i,(r2_score(y_test,y_pred))])
df_scores=pd.DataFrame(scores,columns=['neighbors','R2 score'])
df_scores.sort_values(by='R2 score',ascending=False,inplace=True)
df_scores

,neighbors,R2 score
3,7,0.162803
4,9,0.162501
2,5,0.140374
1,3,0.107408
0,1,-0.258750


1. KNN with StandardScaler scaling

In [ ]:
from sklearn.preprocessing import StandardScaler

scalar = StandardScaler()
ohe=OneHotEncoder(sparse=False,drop='first')

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=25)
ct1=make_column_transformer((ohe,make_column_selector(dtype_include='object')), remainder=scalar,verbose_feature_names_out=False)
ct1=ct1.set_output(transform='pandas')
X_train_scaled=ct1.fit_transform(X_train)
X_test_scaled=ct1.transform(X_test)

In [41]:
# calculate for 1,3,5,7,9
from sklearn.metrics import r2_score
scores=[]
for i in range(1,10,2):
    knn=KNeighborsRegressor(n_neighbors=i)
    knn.fit(X_train_scaled,y_train)
    y_pred=knn.predict(X_test_scaled)
    scores.append([i,(r2_score(y_test,y_pred))])
df_scores=pd.DataFrame(scores,columns=['neighbors','R2 score'])
df_scores.sort_values(by='R2 score',ascending=False,inplace=True)
df_scores

,neighbors,R2 score
2,5,0.658890
1,3,0.656746
3,7,0.636826
4,9,0.623286
0,1,0.569665


1. KNN with minmax scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler

minmax_scalar = MinMaxScaler()
ohe=OneHotEncoder(sparse=False,drop='first')
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=25)

ct1=make_column_transformer((ohe,make_column_selector(dtype_include='object')), remainder=minmax_scalar,verbose_feature_names_out=False)
ct1=ct1.set_output(transform='pandas')

X_train_scaled=ct1.fit_transform(X_train)
x_test_scaled=ct1.transform(X_test)

In [43]:
from sklearn.metrics import r2_score
scores=[]
for i in range(1,10,2):
    knn=KNeighborsRegressor(n_neighbors=i)
    knn.fit(X_train_scaled,y_train)
    y_pred=knn.predict(x_test_scaled)
    scores.append([i,(r2_score(y_test,y_pred))])
df_scores=pd.DataFrame(scores,columns=['neighbors','R2 score'])
df_scores.sort_values(by='R2 score',ascending=False,inplace=True)
df_scores

,neighbors,R2 score
2,5,0.746992
3,7,0.737905
1,3,0.734351
4,9,0.732729
0,1,0.670147


### Best pipeline

In [47]:
scalar = StandardScaler()
ohe=OneHotEncoder(sparse=False,drop='first')

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=25)

In [45]:

ct1=make_column_transformer((ohe,make_column_selector(dtype_include='object')), remainder='passthrough',verbose_feature_names_out=False)

X_train_encoded=ct1.fit_transform(X_train)
X_test_encoded=ct1.transform(X_test)


#Scaling the data
X_train_scaled=scalar.fit_transform(X_train_encoded)
X_test_scaled=scalar.transform(X_test_encoded)

In [48]:
from sklearn.metrics import r2_score
scores=[]
for i in range(1,10,2):
    knn=KNeighborsRegressor(n_neighbors=i)
    knn.fit(X_train_scaled,y_train)
    y_pred=knn.predict(X_test_scaled)
    scores.append([i,(r2_score(y_test,y_pred))])
df_scores=pd.DataFrame(scores,columns=['neighbors','R2 score'])
df_scores.sort_values(by='R2 score',ascending=False,inplace=True)
df_scores

,neighbors,R2 score
2,5,0.783718
3,7,0.778794
1,3,0.772046
4,9,0.768123
0,1,0.688458


### Inferencing

In [51]:
tst=pd.read_csv(r'..\Cases\Medical Cost Personal\tst_insure.csv')

In [52]:
from sklearn.pipeline import Pipeline

sc=StandardScaler()
ohe=OneHotEncoder(sparse=False,drop='first')
knn=KNeighborsRegressor(n_neighbors=5)

ct1=make_column_transformer((ohe,make_column_selector(dtype_include='object')), remainder='passthrough',verbose_feature_names_out=False)
best_model=Pipeline(steps=[('OHE',ct1),("Scale",sc),("KNN",knn)])


In [53]:
best_model.fit(X,y)

Pipeline(steps=[('OHE',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(drop='first',
                                                                sparse=False),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x000002A15DFEB610>)],
                                   verbose_feature_names_out=False)),
                ('Scale', StandardScaler()), ('KNN', KNeighborsRegressor())])

In [54]:
best_model.predict(tst)

array([27374.3606   ,  6923.43421  ,  4923.2172   ,  7746.225252 ,
        6711.63442  ,  7749.9086   ,  1843.61314  ,  2437.65453  ,
       36819.43778  ,  9866.67163  ,  3304.91078  , 27374.3606   ,
        9362.9852   , 22295.73195  ,  6446.1202   , 12201.35051  ,
       13550.3241   , 16329.129206 ,  1859.79761  ,  7106.0190738,
        2054.14964  ,  5739.269666 ,  2037.28478  , 17155.436028 ,
        3722.421466 , 14104.432552 ,  6860.167622 , 22792.585194 ,
        6888.03737  , 30894.566528 ,  4835.75698  , 40915.031192 ,
       12574.459432 , 13896.467678 ,  1963.57     ,  4883.13423  ,
       14769.495438 , 11208.819282 , 19548.61324  ,  9250.472868 ,
       12304.673362 , 21226.36544  , 19840.152966 , 37154.79512  ,
       17155.436028 , 24952.13688  , 37386.98846  , 10770.2092   ,
        8390.300026 ,  4203.17097  ,  9575.48302  ,  5184.436    ,
        8311.08301  , 10889.58527  , 16652.336602 ,  2302.20296  ,
        8844.40044  , 15894.916494 , 37386.98846  ,  8240.314 